In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

sk


### <b>문제 3-1 :  콘텐츠분쟁해결 RAG 시스템 - 간단 실습 가이드</b>
<b>문제 설명</b><br>
: 콘텐츠분쟁해결 사례집을 활용한 법률 자문 RAG 시스템을 순차적으로 구축하는 실습입니다.

In [12]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

print("===> 1. 문서 로딩 → PDF 읽기...")
loader = PyPDFLoader('../data/콘텐츠분쟁해결_사례.pdf')
documents = loader.load()
print(f"  총 {len(documents)}페이지 로드 완료")

print("===> 2. 문서 분할 → 작은 청크로 나누기")
pythontext_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,        # 청크 크기 (한국어 최적화)
    chunk_overlap=300,      # 중복 부분 (맥락 보존)
    separators=[
        "\n【사건개요】",     #  법률 문서 섹션 구분자
        "\n【쟁점사항】",     #  쟁점 부분 구분
        "\n【처리경위】",     #  처리 과정 구분
        "\n【처리결과】",     #  결과 부분 구분
        "\n■", "\n\n", "\n", ".", " ", ""
    ] # 자연스러운 분할을 위한 구분자
)
chunks = text_splitter.split_documents(documents)
print(f"  {len(chunks)}개 청크 생성 완료")
print(f"  평균 청크 길이: {sum(len(chunk.page_content) for chunk in chunks) / len(chunks):.0f}자")

print("===> 3. 벡터화 → 임베딩으로 변환")
pythonembeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",  #  한국어 법률 용어에 최적
    dimensions=1536                  #  고성능 임베딩 차원
)

print("===> 4. 저장 → FAISS 벡터스토어에 저장")
vectorstore = FAISS.from_documents(chunks, embeddings)
print(f" FAISS 벡터스토어 생성 완료 ({len(chunks)}개 벡터)")

print("===> 5. 검색 → 질문과 유사한 문서 찾기")
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}  # 상위 6개 관련 문서 검색
)
print(" Retriever 설정 완료")

print("===> 6. 생성 → LLM으로 답변 생성")
pythonllm = ChatOpenAI(
    model="gpt-4o",                 #  또는 "gpt-4o-mini" (경제적)
    temperature=0.2,                #  법률 조언은 정확성 중시
    max_tokens=2000                 #  충분한 답변 길이
)

# 법률 자문 프롬프트 작성 
pythonprompt_template = """
당신은 콘텐츠 분야 전문 법률 자문사입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요                  
2. 관련 법령이나 조항이 있다면 명시하세요              
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요  
4. 실무적 해결방안을 단계별로 제시하세요          
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요 

전문 법률 조언:"""


prompt = PromptTemplate(
    template=pythonprompt_template,
    input_variables=["context", "question"]
)
print(" 프롬프트 설정 완료")

print("\n ===> 7.  QA 체인 생성...")
pythonqa_chain = RetrievalQA.from_chain_type(
    llm=pythonllm,                        
    chain_type="stuff",            
    retriever=retriever,           
    chain_type_kwargs={"prompt": prompt},  
    return_source_documents=True    
)
print("  RAG 파이프라인 구축 완료!")


print("\n ===> 8.  테스트 질문들")
pythontest_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

print("\n" + "=" * 60)
print(" RAG 시스템 테스트")
print("=" * 60)


print("\n ===> 9.  질문 및 답변 실행")
for i, question in enumerate(pythontest_questions, 1):
    print(f"\n【테스트 {i}/5】")
    print(f" 질문: {question}")
    print(" 답변 생성 중...")
    
    # RAG 실행
    result = pythonqa_chain.invoke({"query": question})
    answer = result["result"]
    source_docs = result["source_documents"]
    
    print(f"\n 답변:")
    print("-" * 50)
    print(answer)
    
    # 참조 문서 정보
    print(f"\n 참조 문서:")
    for j, doc in enumerate(source_docs[:3], 1):
        page = doc.metadata.get('page', 'N/A')
        preview = doc.page_content[:80].replace('\n', ' ')
        print(f"   {j}. 페이지 {page}: {preview}...")
    
    print("\n" + "-" * 40)


===> 1. 문서 로딩 → PDF 읽기...
  총 109페이지 로드 완료
===> 2. 문서 분할 → 작은 청크로 나누기
  134개 청크 생성 완료
  평균 청크 길이: 571자
===> 3. 벡터화 → 임베딩으로 변환
===> 4. 저장 → FAISS 벡터스토어에 저장
 FAISS 벡터스토어 생성 완료 (134개 벡터)
===> 5. 검색 → 질문과 유사한 문서 찾기
 Retriever 설정 완료
===> 6. 생성 → LLM으로 답변 생성
 프롬프트 설정 완료

 ===> 7.  QA 체인 생성...
  RAG 파이프라인 구축 완료!

 ===> 8.  테스트 질문들

 RAG 시스템 테스트

 ===> 9.  질문 및 답변 실행

【테스트 1/5】
 질문: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
 답변 생성 중...

 답변:
--------------------------------------------------
온라인 게임에서 시스템 오류로 인해 아이템이 사라진 경우, 게임회사가 복구를 거부하는 상황에 대한 법률 조언을 드리겠습니다. 제시된 사례들을 근거로 하여 답변을 구성하였습니다.

1. **사례 분석**:
   - **사례 1**: 시스템 오류로 아이템이 소멸되었으나, 계정 명의자가 아니어서 복구가 거부된 사례입니다. 이 경우, 계정 명의자가 직접 복구 요청을 해야 한다는 점이 강조되었습니다.
   - **사례 23**: 시스템 오류로 인한 게임머니 손실을 주장했으나, 시스템 오류가 발생하지 않았다는 게임사의 입장과 사용 내역을 근거로 복구가 거부된 사례입니다.
   - **사례 13**: 보안 서비스 미작동으로 인한 아이템 도난에 대해, 게임사가 도난을 인정하고 복구를 권고받은 사례입니다.

2. **관련 법령**:
   - 민법 제250조 및 제251조는 도품이나 유실물에 대한 반환 청구에 관한 규정을 두고 있습니다. 그러나 이는 주로 물리적 재산에 대한 것이